# Análise dos Tópicos 

Esse notebook possui os códigos para a análise dos tópicos extraídos dos diários oficiais. 

## Imports Necessários

In [1]:
# Código para poder usar os módulos implementados nesse repositório no notebook 
import sys, os
path_module = os.path.abspath('../modules/')
if path_module not in sys.path:
    sys.path.append(path_module)

In [2]:
# imports do projeto

from utils.process_gazette import ProcessGazette
from preprocess.pre_process_text import PreProcessText
from nlp.extract_topics import ExtractTopics
from nlp.extract_topicsLLM import ExtractTopicsLLM
from nlp.ner import Ner
from nlp.sentiment import Sentiment
from nlp.summarizer import Summarizer
from nlp.classifier import Classifier

## Extraindo de multiplas gazetas

In [3]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [4]:
cont = 1 #len(all_pages)

In [5]:
all_topics = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")
for gazette in range(0,cont):
    all_topics[gazette] = {}
    for page, text in all_pages[gazette].items():
        topics_[page] = {'text': text, 'topics': []}
        tokens_ = ppt.process_text(topics_[page]['text'])
        if len(tokens_) > 5 and page:
            topics_[page]['tokens'] = tokens_
            
            all_topics[gazette][page] = topics_[page]

c:\Users\erick\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Escolha da gazeta e da página
gazette = 0
page = 4

## Amplamente difundido

### Bertopic

In [14]:
extractor = ExtractTopics(model = 'default')
#extractorGpt = ExtractTopics(model = 'gpt')
extractorZeroshot = ExtractTopics(model = 'zeroshot')
extractorMaritalk = ExtractTopics(model = 'maritalk')

all_topics[gazette][page]['topics'], all_topics[gazette][page]['topics_info'] = extractor.extract_topics(all_topics[gazette][page]['tokens'])
#all_topics[gazette][page]['topicsGpt'], all_topics[gazette][page]['topics_infoGpt'] = extractorGpt.extract_topics(all_topics[gazette][page]['tokens'])
all_topics[gazette][page]['topicsZeroshot'], all_topics[gazette][page]['topics_infoZeroshot'] = extractorZeroshot.extract_topics(all_topics[gazette][page]['tokens'])
all_topics[gazette][page]['topicsMaritalk'], all_topics[gazette][page]['topics_infoMaritalk'] = extractorMaritalk.extract_topics(all_topics[gazette][page]['tokens'])


In [15]:
all_topics[gazette][page]['topics_info'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,104,-1_acordo_objetivo_compor_finalidade,"[acordo, objetivo, compor, finalidade, convida...","[acordo, compor, acordo]"
1,0,23,0_urbanismo___,"[urbanismo, , , , , , , , , ]","[municipal, urbanismo, urbanismo]"
2,1,19,1_entidade_instituicao_gabinete_desenvolvimento,"[entidade, instituicao, gabinete, desenvolvime...","[entidade, entidade, entidade]"
3,2,18,2_secretaria_geral_secretario_,"[secretaria, geral, secretario, , , , , , , ]","[secretario-geral, Secretaria, Secretaria]"
4,3,18,3_situacao_indireto_calamidade_conforme,"[situacao, indireto, calamidade, conforme, sim...","[indireto, indireto, situacao]"


In [16]:
all_topics[gazette][page]['topics_infoMaritalk'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,67,-1_coordenacao_municipio_membro_objetivo,"[coordenacao, municipio, membro, objetivo, res...","[coordenacao, coordenacao, coordenacao]"
1,0,88,0_autorizar_privado_representante_pobreza,"[autorizar, privado, representante, pobreza, c...","[privado, autorizar, privado]"
2,1,44,1_operacao_prefeito_administracao_promocao,"[operacao, prefeito, administracao, promocao, ...","[operacao, operacao, operacao]"
3,2,38,2_acordo_entidade_enfrentamento_situacao,"[acordo, entidade, enfrentamento, situacao, mo...","[entidade, acordo, acordo]"
4,3,29,3_integracao_trabalho_reuniao_requisitar,"[integracao, trabalho, reuniao, requisitar, du...","[integracao, integracao, integracao]"


In [17]:
all_topics[gazette][page]['topics_infoZeroshot'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,0,37,Dispensas de Licitação,"[publico, informacao, resultado, modificacao, ...","[publico, publico, Publico]"
1,1,7,Contratações Emergenciais,"[emergencia, , , , , , , , , ]","[emergencia, emergencia, Emergencia]"
2,2,5,Aquisições Diretas,"[diretor, direto, direito, diretriz, , , , , , ]","[direto, diretor, diretor]"
3,3,62,-1_membro_tomada__,"[membro, tomada, , , , , , , , ]","[membro, membro, membro]"
4,4,32,0_indicar_indireto_inspetor_,"[indicar, indireto, inspetor, , , , , , , ]","[indicar, indicar, indicar]"


### NER

In [7]:
ner = Ner()
all_topics[gazette][page]['ner_tokens'] = ner.extract_entities_tokens(all_topics[gazette][page]['tokens'])
print(all_topics[gazette][page]['ner_tokens'])
all_topics[gazette][page]['ner_text'] = ner.extract_entities_text(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['ner_text'])

[(periodo, 'PER'), (gestao convenio, 'LOC'), (modificacao, 'LOC'), (instituicao comite, 'ORG'), (Educacao, 'LOC'), (Camara Municipal, 'PER'), (Ministerio Publico, 'ORG'), (Brasil, 'LOC'), (Centro, 'LOC'), (Guarda, 'LOC'), (Secretaria Municipal, 'LOC'), (Secretaria Municipal, 'LOC'), (Centro, 'LOC'), (Centro, 'LOC')]
[(Capítulo, 'MISC'), (Município de Salvador, 'LOC'), (Decreto, 'MISC'), (Lei, 'MISC'), (CAPÍTULO V DO COMITÊ DE INTEGRAÇÃO INSTITUCIONAL  Art. 15, 'MISC'), (Comitê de Integração Institucional, 'ORG'), (Município de Salvador, 'LOC'), (Covid-19, 'LOC'), (Comitê de Integração Institucional, 'ORG'), (I - Prefeito Municipal, 'MISC'), (II - Vice-Prefeito, 'MISC'), (Casa Civil, 'PER'), (IV - Chefe de Gabinete do Prefeito, 'MISC'), (V - Procurador-Geral do Município, 'MISC'), (Secretário Municipal da Saúde, 'PER'), (Secretário Municipal da Fazenda, 'LOC'), (Secretário Municipal de Promoção Social e Combate à Pobreza, 'PER'), (Secretário Municipal de Educação, 'PER'), (Secretário Mu

### Sentiment

In [13]:
sentiment = Sentiment()
all_topics[gazette][page]['sentiment'] = sentiment.sentiment_analyze_spacy(all_topics[gazette][page]['text'])
all_topics[gazette][page]['sentiment']

'positive'

## LLMs

### Classifier

In [20]:
classifier = Classifier()
all_topics[gazette][page]['classe'] = classifier.classify(all_topics[gazette][page]['text'])
all_topics[gazette][page]['classe']

' Atos Administrativos'

### Extract Topics LLM

In [7]:
extractTopicsLLM = ExtractTopicsLLM()
all_topics[gazette][page]['topics'] = extractTopicsLLM.extract_topics(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['topics'])

  ```python
    {
        "Lei Municipal N° 7.6114": {
            "Description": "Institui normas excepcionais para o enfrentamento da pandemia de Covid-19 no âmbito do Município de Salvador.",
            "Frequency": 1
        },
        "Capítulo V - Comitê de Integração Institucional": {
            "Description": "Define a composição e competências do Comitê de Integração Institucional.",
            "Frequency": 1
        },
        "Art. 15 - Composição do Comitê": {
            "Description": "Lista os membros do Comitê de Integração Institucional.",
            "Frequency": 1
        },
        "Art. 16 - Composição do Comitê - Detalhamento": {
            "Description": "Detalha a composição do Comitê de Integração Institucional, incluindo membros do Executivo e Legislativo Municipais e outras autoridades.",
            "Frequency": 1
        },
        "Art. 17 - Centro de Coordenação de Operações": {
            "Description": "Descreve as funções do Centro de Coordenação 

### NER

In [10]:
ner = Ner(model = 'maritalk')
all_topics[gazette][page]['ner_mari'] = ner.extract_entities_maritalk(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['ner_mari'])

  ```python
{
    "Person": ["Prefeito Municipal", "Vice-Prefeito", "Chefe da Casa Civil", "Chefe de Gabinete do Prefeito", "Procurador-Geral do Município", "Secretário Municipal da Saúde", "Secretário Municipal da Fazenda", "Secretário Municipal de Promoção Social e Combate à Pobreza", "Secretário Municipal de Educação", "Secretário Municipal de Infraestrutura", "Secretário Municipal de Gestão", "Secretário Municipal de Mobilidade", "Secretário Municipal de Desenvolvimento e Urbanismo", "Secretário Municipal de Ordem Pública", "Secretário Municipal de Comunicação", "Secretário-Geral de Articulação Comunitária e Prefeituras-Bairro", "Controlador-Geral do Município"],
    "Organization": ["Prefeitura Municipal de Salvador", "Tribunal de Contas dos Município-TCM do Estado da Bahia"],
    "Location": ["Salvador"],
    "Date": ["19 de março de 2020", "período da pandemia"],
    "Money": ["verba própria do orçamento vigente"]
    # ... outras entidades conforme necessário
}
```


### Sentiment

In [12]:
sentiment = Sentiment(model = 'maritalk')
all_topics[gazette][page]['sentiment_mari'] = sentiment.sentiment_analyze_maritalk(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['sentiment_mari'])

  Negativo.


### Summarizer

In [14]:
summarizer = Summarizer()
all_topics[gazette][page]['resumo'] = summarizer.summary(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['resumo'])

 Resumo:
A Lei Municipal no 7.6114 institui medidas temporárias e excepcionais para o enfrentamento da pandemia de COVID-19. O Artigo 13 determina que ações de colaboração e fomento, acordos de cooperação, termos de parceria, contratos de repasse, contratos de gestão e convênios com vencimento a partir de 19 de março de 2020 podem ser prorrogados por até 3 meses após o fim da emergência, mantendo-se as condições originais e seguindo procedimentos legais. O Artigo 14 autoriza o estabelecimento de regras excepcionais para a prestação de contas desses ajustes, desde que relacionadas à pandemia e em conformidade com a legislação vigente.

O Capítulo V institui o Comitê de Integração Institucional, com a finalidade de centralizar decisões, integrar e alinhar ações governamentais de enfrentamento da pandemia, prevenir, dar assistência à população e minimizar impactos da emergência e do estado de calamidade pública. O Comitê é composto por diversas autoridades, incluindo o Prefeito, Vice-Pref

#### Este foi pensado para ser usado em licitações

In [15]:
summarizer = Summarizer()
all_topics[gazette][page]['licitaçãoInfo'] = summarizer.extract_info(all_topics[gazette][page]['text'])
print(all_topics[gazette][page]['licitaçãoInfo'])

  ```python
{
    "Licitação 1": {
        "Empresa": "Prefeitura Municipal de Salvador",
        "Valor": "Valor não mencionado",
        "Objetivos": "Coordenar e acompanhar de perto a efetivação das providências a cargo da Prefeitura Municipal no enfrentamento do Covid-19"
    },
    "Licitação 2": {
        "Empresa": "Centro de Coordenação de Operações",
        "Valor": "Valor não mencionado",
        "Objetivos": "Articulação, com entes públicos e privados, ações adequadas e avaliar seus respectivos impactos"
    }
}
```
